# Topic Modeling mit scikit-learn

Du kennst jetzt schon die Funktionsweise von Topic Modeling. In diesem Notebook schauen wir uns jetzt einmal an, wie du mithilfe der Clusteringmethode KMeans aus einem Korpus die relevanten Themen extrahieren kannst.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## Korpus


Um Themen zu extrahieren, benötigen wir zuerst ein Korpus. Da trifft es sich doch gut, dass im Python-Modul scikit-learn bereits ein Textkorpus zur Verfügung gestellt wird. Es handelt sich hier um den Datensatz `20newsgroups`, in dem englischsprachige Beiträge aus 20 verschiedenen Online-Foren der 90er Jahre gesammelt wurden. Diese decken ein breites Spektrum an Diskussionsthemen ab, z. B. Sport, Politik und Wissenschaft. Wir laden den Datensatz mit dem Befehl `fetch_20newsgroups` aus dem Modul `sklearn.datasets`. Bei der Funktion können wir mit `subset=''` bereits angeben, welche Foren wir betrachten wollen, in diesem Fall alle.

Mit dem Datensatz kann auch eine Klassifikation durchgeführt werden, bei denen die Texte zu den speziellen Foren zugeteilt werden sollen. Daher existiert im Datensatz eine Zielvariable, die wir aber nicht benötigen. Die Option `return_X_y=True` gibt uns die Daten getrennt nach Beobachtungen und Zielvariable zurück, und der Unterstrich `_` gibt an, dass die Zielvariable nicht gespeichert werden muss. Unsere Textdokumente sind jetzt also in `documents` gespeichert.


In [ ]:
# Laden des Datensatzes
documents, _ = fetch_20newsgroups(subset='all', return_X_y=True)

Eine Beispielnachricht sieht dann so aus:

In [ ]:
#Gebe den Beitrag Nummer 30 aus
print(documents[30])

Der obere Teil ist die Kopfzeile. Hier findest du unter anderem den Benutzernamen des oder der Beitrag-Ersteller:in. Danach kommt der Textkörper. An den spitzen Klammern nach links erkennst du Zitate, also Textteile, die in anderen Beiträgen vorkommen und auf die sich der aktuelle Beitrag bezieht. Den Abschluss macht eine Fußzeile, in diesem Fall eine Signatur. Da es uns nur um die Themen der Beiträge selber geht, sollten wir auch alle Kopfzeilen, Fußzeilen und Zitate aus den Beiträgen entfernen. Wir laden die Daten also noch einmal, entfernen dieses Mal aber mit der `remove`-Option der `fetch_20newsgroups`-Methode die ungewünschten Nachrichtenbestandteile.

In [ ]:
#Noch mal laden, dieses Mal ohne headers, footers und quotes
documents, _ = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), return_X_y=True)

## Bereinigung und Tf-idf-Vectorizer

Jetzt haben wir ein Korpus an Dokumenten, mit dem wir aber in seiner jetzigen Form noch nichts anfangen können. Um die Themen der Dokumente erkennen zu können, brauchen wir eine geeignete Darstellung der Dokumente. Dafür können wir den TF-IDF-Vectorizer aus dem Modul scikit-learn verwenden.

Der Vectorizer erstellt, wie der Name bereits vermuten lässt, aus den jeweiligen Wörtern eines Dokuments einen Vektor mit den entsprechenden tf-idf-Werten. Die Vektoren der einzelnen Dokumente werden dann zu einer Matrix zusammengefasst. Zur Erinnerung: Tf-idf steht für “term frequency – inverse document frequency” und ist ein Maß für die Relevanz eines Wortes innerhalb eines Korpus. Je häufiger ein Wort in einem Dokument und je seltener dieses Wort im gesamten Korpus auftaucht, desto höher ist der tf-idf-Wert dieses Wortes im betrachteten Dokument. So können wir auf einen Blick die relevantesten Wörter der jeweiligen Dokumente erkennen.

Als Erstes sollten wir unseren Datensatz aber bereinigen. Zum Glück sind im Vectorizer bereits viele Vorverarbeitungsschritte integriert. Mit dem Parameter `vocabulary` können wir ein Wörterbuch übergeben. Mit `vocabulary=None` weisen wir den Vectorizer an, sich sein eigenes Wörterbuch zu erstellen. Wir erstellen unsere spätere tf-idf-Matrix nur mit den Wörtern des Wörterbuchs.

Jetzt müssen unsere Texte noch tokenisiert werden. Durch `analyzer=’word’` geben wir an, dass unsere Tokens aus ganzen Wörtern bestehen sollen. Der Parameter `token_pattern` gibt an, wie Wörter aussehen müssen, damit sie als Token gezählt werden. Der Buchstabensalat dahinter ist ein sogenannter regulärer Ausdruck. Keine Sorge, du musst als Anfänger:in nicht wissen, was das ist, oder selber reguläre Ausdrücke lesen und schreiben können. Wenn du dich allerdings näher mit Textverarbeitung beschäftigen möchtest, sind reguläre Ausdrücke ein nützliches Hilfsmittel. Der von mir benutzte reguläre Ausdruck gibt vereinfacht gesagt an, dass nur Wörter aus mindestens zwei Buchstaben als Token betrachtet werden sollen. So werden also u. a. Ziffern und Sonderzeichen aussortiert.

Als Nächstes wandeln wir mit `lowercase=True` alle Zeichen unseres Textes in Kleinbuchstaben um. Zur Erinnerung: Unsere Texte sind Strings, und alle Zeichen eines Textes sind mit ASCII kodiert. Großbuchstaben haben einen anderen ASCII-Code als die dazugehörigen Kleinbuchstaben. Nachdem unsere Texte in kleingeschriebene Wörter aufgeteilt sind, können wir die Stoppwörter entfernen. Durch `stopwords=’english’` wird eine integrierte englische Liste verwendet. Das Entfernen von zu vielen, zu wenigen oder falschen Wörtern kann aber die Ergebnisse deiner Analysen stark beeinflussen, also solltest du dir immer gut überlegen, welche Stoppwörter für deinen Kontext gut geeignet sind! Durch den Parameter `stopwords` kannst du auch eine eigene Liste an den Vectorizer übergeben.

Wir erstellen also den Vectorizer durch den Befehl `TfidfVectorizer` mit den eben genannten Parameteroptionen. Wir wenden ihn an, indem wir die Methode `fit_transform` mit unserem Korpus aufrufen.


In [ ]:
#in Kleinbuchstaben umwandeln, Stopwörter entfernen, Tokenisierung per Wort, Sonderzeichen etc entfernen
vectorizer = TfidfVectorizer(vocabulary=None, analyzer='word', token_pattern=r"(?u)\b[a-zA-Z]{2,}\b",  lowercase=True, stop_words='english' )

#unseren Korpus in die gewünschte Form umwandeln
X = vectorizer.fit_transform(documents)

## Themen extrahieren

Da wir jetzt ein Korpus in geeigneter Darstellung haben, können wir endlich unsere Themen extrahieren. Eine Möglichkeit, dies zu tun, ist, mit dem Clustering-Algorithmus KMeans. KMeans nimmt die tf-idf-Vektoren der Dokumente, um ähnliche Dokumente zu clustern, sprich zu gruppieren. Jedes Dokumentencluster entspricht dann einem Oberthema. Die Clusteranzahl `k` ist dabei ein Maß, wie differenziert die gefundenen Themen dabei sein sollen. So werden sich bei einem kleinen `k` wahrscheinlich alle sportbezogenen Beiträge dasselbe Cluster teilen, während bei einem sehr großen `k` womöglich Unterschiede zwischen den jeweiligen Sportarten gemacht werden. In unserem Beispiel nennen wir die Clusteranzahl `topic_number`, setzen sie auf 15 und erstellen und trainieren unser Modell. Aber probiere hier gerne einmal herum!


In [ ]:
#Unterschiedliche Anzahl Cluster ausprobieren
topic_number = 15
km = KMeans(n_clusters=topic_number)

In [ ]:
#Modell trainieren
km.fit(X)

Nachdem die Cluster erstellt sind, möchten wir feststellen, welche Themen unser Modell denn identifiziert hat. Das geht natürlich nicht mit Worten, sprich KMeans gibt nicht Begriffe wie „Sport” oder „Politik” zurück. Wir behelfen uns also, indem wir uns die wichtigsten Wörter eines Dokumentenclusters ausgeben lassen. Durch den Befehl `get_feature_names_out()` erhalten wir die Features unserer Dokumente. Das sind hier die Wörter des Wörterbuches, insgesamt stolze 86.567. Außerdem definieren wir uns eine Variable `top_n`, die angibt, wie viele Wörter pro Cluster wir uns ausgeben lassen.

In [ ]:
# Extrahiere für jedes Cluster die Top-Terme basierend auf den Clusterzentren
terms = vectorizer.get_feature_names_out()
#Lass dir die Größe des Wörterbuches anzeigen
print(terms.shape)
# Anzahl der Top-Terme pro Cluster, die wir später ausgegeben bekommen möchten
top_n = 10

 ### Optional: Erklärung der Hilfsfunktion

Wir definieren uns eine Methode zum Ausgeben der Top-Terme. Du kannst eine eigene Methode definieren, indem du den Befehl `def` verwendest und danach den gewünschten Namen und die Parameter der Methode angibst. Unsere Methode soll `print_top_terms` heißen und als Parameter einmal die Terme aus dem Wörterbuch erhalten (`terms`), dann die Zentroide der gefundenen Cluster, sprich ihre Mittelpunkte (`unsorted_centroids`) und als letztes die Anzahl der Cluster (`topic_number`). Das sind nur die gewählten Namen im Rahmen der Methode, wir hätten diese Parameter also auch `x`, `y` und `z` nennen können.

Damit wir die wichtigsten Terme der einzelnen Cluster bestimmen können, müssen wir uns die Terme angucken, die die höchste Gewichtung in jedem Cluster haben. Dafür müssen wir erst einmal verstehen, was ein Cluster-Mittelpunkt in unserem Fall überhaupt aussagt.


Angenommen, wir haben das Wörterbuch `[Katze, Kaffee, Auto, Puppe, Berg]`, drei Cluster und wollen das jeweils wichtigste Wort der Dokumente im Cluster herausfinden (`top_n = 1`). Dann können die Clustermittelpunkte zum Beispiel so aussehen:

```
[[0.1, 0.5, 0.3, 0.2, 0.7],  # Cluster 1
 [0.3, 0.2, 0.8, 0.5, 0.1],  # Cluster 2
 [0.6, 0.4, 0.2, 0.1, 0.3]]  # Cluster 3
```

Bei unserem Textbeispiel gilt also: Es gibt eine Zeile für jedes Cluster und eine Spalte für jedes Wort des Wörterbuches. Die Zahlen geben die tf-idf-Scores der Wörterbuchworte des Mittelpunktes an. Je höher also der numerische Wert bei einem Wort, desto wichtiger ist dieses Wort (Feature) für das jeweilige Cluster. Damit wir die wichtigsten Wörter ausgeben können, können wir unsere Mittelpunke also sortieren. Wenn wir jetzt nur mit der `sort`-Methode die Werte sortieren, verlieren wir aber die Zuordnung: Aus `[0.1, 0.5, 0.3, 0.2, 0.7]` würde `[0.1, 0.2, 0.3, 0.5, 0.7]` werden und wir wüssten nicht mehr, welches Wort denn jetzt 0.5 beigetragen hat. Stattdessen verwenden wir die Methode `argsort`. Diese Methode gibt nicht die Werte, sondern die Indizes der Elemente in aufsteigender Reihenfolge zurück. Für den Zentroid `[0.1, 0.5, 0.3, 0.2, 0.7]` gibt `argsort` folgendes zurück: `[0, 3, 2, 1, 4]`.

Am wenigsten wichtig war also das Feature an Stelle 0 (`Katze`, denk daran, dass wir mit 0 anfangen zu zählen!), am zweitwichtigsten das an Stelle 3 (`Puppe`) usw..

Für unsere Beispiel-Cluster ergibt sich mit `argsort` also:

```
[[0, 3, 2, 1, 4],  # Cluster 1
 [4, 1, 0, 3, 2],  # Cluster 2
 [3, 2, 4, 1, 0]]  # Cluster 3`
```


Als nächstes sammeln wir unsere gefundenen Top-Terme in einer Liste. Um alle Cluster durchzugehen, benutzen wir eine sogenannte `for`-Schleife. Eine Schleife wiederholt unsere Anweisungen solange, bis eine bestimmte Bedingung erfüllt ist. Für unsere `for`-Schleife haben wir eine Zählvariable, hier `i`, die von 0 anfängt und nach jedem Schleifendurchlauf um eins erhöht wird. Bevor die Anweisungen der Schleife erneut ausgeführt werden, wird geprüft, ob die Zählvariable die Abbruchhöhe erreicht hat, hier `topic_number`, also die Anzahl an Clustern. Wenn das nicht der Fall ist, werden die Anweisungen der Schleife erneut ausgeführt, ansonsten wird das Programm mit den Anweisungen nach der Schleife weitergeführt. Du erkennst, welche Anweisungen zur Schleife gehören, weil die entsprechenden Programmzeilen eingerückt sind.

Zusammengefasst bedeutet `for i in range(topic_number)` also, dass die nachfolgenden Anweisungen `topic_number`-oft durchgeführt werden sollen.

Aber was genau tun wir in dieser Schleife eigentlich? In jedem Schleifendurchlauf steht `i` für den gerade betrachteten sortierten Zentroiden. Mit `sorted_centroids[i, ::-1][:top_n]]` wählen wir die hintersten `top_n` Einträge von Zentroid `i` aus. Zur Erinnerung: In den sortierten Zentroiden stehen die Indizes der Top-Terme ganz hinten, weil sie von unwichtig nach wichtig sortiert wurden.
Falls dir die Schreibweise mit den Doppelpunkten nichts (mehr) sagt, schau dir doch mal einen Beitrag zu Slicing in Python an, zum Beispiel den hier: https://statistikguru.de/python/slicing-in-python.html

Für jeden Index aus den hinteren `top_n`-Indizes (`for index in sorted_centroids[i, ::-1][:top_n]`) sucht das Programm jetzt heraus, welches Wort des Wörterbuches zu diesem Index gehört (`terms[index]`). Die eckigen Klammern um den gesamten Ausdruck zeigen an, dass diese Wörter in eine Liste geschrieben werden, die dann unter `top_terms_for_cluster` abgespeichert wird. Danach wird diese Liste an unsere `top_terms`-Liste angehängt (`top_terms.append(top_terms_for_cluster)`) und der nächste Schleifendurchlauf startet. Dabei wird die Liste `top_terms_for_cluster` mit den Top-Termen des nächsten Clusters überschrieben, sodass wir immer nur die Top-Terme des aktuell betrachteten Clusters anhängen. In unserem Beispiel wird also im ersten Schleifendurchlauf `Berg` in die Liste der Top-Terme des ersten Clusters aufgenommen, dann `Auto` für das zweite Cluster und `Katze` für das dritte Cluster, weil jeweils Index 4, 2 bzw. 0 als letzter Eintrag in den sortierten Clustermittelpunkten standen.

In der nächsten `for`-Schleife (`for i, terms_for_cluster in enumerate(top_terms)`) gehen wir jetzt alle Cluster und alle Wörter aus der Liste `top_terms` durch und geben sie aus. Der Befehl `', '.join(terms_for_cluster)` verknüpft dabei alle Terme aus dem jeweiligen Listeneintrag eines Clusters mit einem Komma.

In [ ]:
#Hilfsfunktion, muss nicht verstanden werden
def print_top_terms(terms, unsorted_centroids,topic_number):
    # Sortieren der Clusterzentren nach Gewicht
    sorted_centroids = unsorted_centroids.argsort()
    top_terms = []
    for i in range(topic_number):
        top_terms_for_cluster = [terms[index] for index in sorted_centroids[i, ::-1][:top_n]]
        top_terms.append(top_terms_for_cluster)

    for i, terms_for_cluster in enumerate(top_terms):
        print("Top-Terme für Cluster " + str(i+1) + ": " + ', '.join(terms_for_cluster))

## Weiter geht's!

Für das Ausgeben der Wörter nutzen wir die selbstgeschriebene Methode print_top_terms, die als Parameter die Wörterbuch-Wörter, die Clusterzentren und die Anzahl an Themen/Cluster erhält und dann die jeweiligen wichtigsten Wörter ausgibt. Die Befehle der Hilfsmethode müssen nicht von dir verstanden werden, nur die grobe Funktionsweise.

Die Clusterzentren sind in der Variable `cluster_centers_` des KMeans-Modell `km` gespeichert, daher kann mit `km.cluster_centers_` darauf zugegriffen werden. Es handelt sich hier um eine Matrix, bei der jede Zeile den Mittelpunkt des jeweiligen Clusters angibt und die Spalten die Koordinaten im Raum sind. Wir haben 15 Cluster und 86567 Wörter im Wörterbuch, also einen 86567-dimensionalen Raum, in dem sich die Cluster befinden. Da wir die Dokumente in tf-idf-Vektoren umgewandelt haben, können wir durch die Koordinaten der Cluster-Mittelpunkte feststellen, welche Wörter für jedes Cluster am wichtigsten sind. Diese Wörter erkennen wir daran, dass ihre tf-idf-Werte im angegebenen Mittelpunkt am höchsten sind.

Und voila: Folgende Cluster mit den jeweiligen wichtigsten Wörtern hat unser Modell gefunden!

In [ ]:
#Ausgabe der top-Terme mithilfe der zuvor definierten Hilfsfunktion
print_top_terms(terms, km.cluster_centers_, topic_number)

Wie genau die Cluster aufgebaut sind, ändert sich natürlich von Training zu Training, daher können sich deine Begriffe von denen im Video unterscheiden. Wahrscheinlich beschäftigt sich aber mindestens ein Cluster wohl mit Computern, eins mit Sport, eins mit Weltraum, eins mit Religion, und eins mit Hilfegesuchen.

## Abschluss

Wir haben nur eine sehr grobe Bereinigung des Datensatzes durchgeführt. So haben wir unter anderem keine URLs entfernt, was zum Auftreten von typischen URL-Bestandteilen wie com, edu und gov geführt hat. Außerdem haben wir keine Auflösung von Verkürzungen vorgenommen, sodass auch Terme wie don (für don’t) und ve (für das verkürzte have) in unserer Themenliste auftauchen. Wenn wir uns einen kleinen Ausschnitt des erzeugten Wörterbuchs anzeigen lassen, wird auch schnell klar, dass eine Rechtschreibkorrektur und eine Lemmatisierung sicher ganz sinnvoll gewesen wären. Probier‘ doch selber einmal aus, das Ergebnis zu verbessern!


In [ ]:
print(terms[977:996])